# FakeNews Detection

### Importing all the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential


In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\anaconda3\share\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
import pandas as pd
import numpy as np

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

### Loading the Data

In [4]:
data = pd.read_csv(r'E:\1. Tech\Machine Learning Projects\FakeNews_classification\train.csv')
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


### Removing The NAN values from the data

In [5]:
data = data.dropna()
data.reset_index(inplace = True)

### Extracting Dependent and Independent variables from data

In [6]:
x = data.drop('label', axis = 1)
y = data['label']

### Checking shape of the data

In [7]:
print(x.shape)
print(y.shape)

(18285, 5)
(18285,)


### Preprocessing the Text

In [9]:
messages = x.copy()
ps = PorterStemmer()
corpus = []

for i in range(len(messages)):
    review = re.sub("[^a-zA-Z]", " ", messages['title'][i])
    review = review.lower().split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

### One Hot Encoding

###### Way of representing text into numbers

In [10]:
vocab_size = 5000
onehot_repr = [one_hot(words, vocab_size) for words in corpus]
onehot_repr

[[2168, 3615, 981, 330, 893, 319, 1896, 361, 3910, 3215],
 [3747, 2446, 2433, 4382, 4574, 2168, 682],
 [3902, 2136, 2326, 4197],
 [3825, 934, 4653, 4825, 1947, 534],
 [1024, 4574, 3061, 3241, 2710, 2357, 4574, 1128, 2082, 91],
 [1374,
  4991,
  3047,
  4608,
  2002,
  4992,
  2234,
  2311,
  3833,
  2205,
  630,
  2527,
  1723,
  1786,
  682],
 [2102, 1559, 2129, 3255, 4404, 3595, 1740, 3765, 2136, 4970, 3139],
 [2353, 1912, 3244, 1125, 1128, 1477, 4992, 3356, 2136, 4970, 3139],
 [1467, 3890, 2917, 3946, 2718, 570, 1359, 2933, 4992, 3283],
 [2584, 2499, 4565, 2774, 2555, 3933, 1407, 4155],
 [3125, 4690, 1304, 4672, 1674, 912, 3598, 1731, 3943, 749, 3879],
 [4825, 1530, 893, 570, 4992, 1128],
 [1579, 4015, 4632, 159, 4110, 160, 3925, 4376, 4166],
 [2577, 1104, 1324, 3530, 3019, 2063, 2563, 2136, 4970, 3139],
 [1514, 2968, 2585, 1185, 1137, 2136, 4970, 3139],
 [3308, 428, 1103, 842, 2768, 3529, 1491, 1817, 2823, 4406],
 [3332, 3276, 2446],
 [1785, 3682, 189, 931, 4992, 1730, 1416, 682],


### Appying Pad sequences to make sentence length Equal

In [11]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  361 3910 3215]
 [   0    0    0 ... 4574 2168  682]
 [   0    0    0 ... 2136 2326 4197]
 ...
 [   0    0    0 ... 2136 4970 3139]
 [   0    0    0 ... 3012 1403  604]
 [   0    0    0 ... 1584 1036 2680]]


### Creating the model for Training Set

In [22]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size= 0.33, random_state=42)

In [25]:
model.fit(x_train, y_train, validation_data= (x_test, y_test), epochs=10, batch_size = 64)

Epoch 1/10
192/192 [==============================] - 7s 21ms/step - loss: 0.3266 - accuracy: 0.8504 - val_loss: 0.1969 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 3s 18ms/step - loss: 0.1409 - accuracy: 0.9444 - val_loss: 0.2004 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 4s 18ms/step - loss: 0.1011 - accuracy: 0.9633 - val_loss: 0.2167 - val_accuracy: 0.9163
Epoch 4/10
192/192 [==============================] - 4s 18ms/step - loss: 0.0737 - accuracy: 0.9733 - val_loss: 0.2657 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0533 - accuracy: 0.9812 - val_loss: 0.3203 - val_accuracy: 0.9133
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0401 - accuracy: 0.9865 - val_loss: 0.3074 - val_accuracy: 0.9118
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0293 - accuracy: 0.9909 - val_loss: 0.4134 - val_accuracy: 0.9097

### Performance Measures and Accuracy

In [27]:
y_pred = model.predict(x_test)
y_pred = np.round(y_pred).astype(int)
confusion_matrix(y_test, y_pred)


189/189 [==============================] - 1s 5ms/step


array([[3096,  323],
       [ 201, 2415]], dtype=int64)

In [29]:
accuracy_score(y_test, y_pred)

0.9131731565865783